In [5]:
import os
for file_name in os.listdir(
        r"C:\Users\hfu\Desktop\Wissenschaftliche Arbeiten Ordner\Wissenschaftliche Arbeiten Ordner"
    ):
    print(file_name)

0_Austausch ANTS
1_Organisatorisches und Aufgabenstellung
2_Hinweise, Leitfäden und Arbeitssicherheit
3_Literatur
4_Experimentelle Daten
5_Dokumentation
6_Schriftliche Ausarbeitung
7_Präsentation
WICHTIG_Read_Me_FAQ_wissenschaftliche_Arbeiten.docx


In [1]:
from models import seg_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary
from fvcore.nn import FlopCountAnalysis


subprocess.run('nvidia-smi', shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load('configs/default.yaml')
cmd_config = {
    'usr_config': 'configs/seg_boltzmannT01_bin6.yaml',
    'datasets': 'shapenet_AnTao350M',
    'wandb': {'name': '2024_04_16_01_26_Shapenet_Token_Std_logmean_1'},
    'test': {'ddp': {'which_gpu': [3]}}
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f'configs/datasets/{config.datasets}.yaml')
dataset_config = OmegaConf.create({'datasets': dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(f'{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest')
    if config.test.suffix.enable:
        local_path = f'./artifacts/{config.wandb.name}_{config.test.suffix.remark}'
    else:
        local_path = f'./artifacts/{config.wandb.name}'
    # artifact.download(root=local_path)
else:
    raise ValueError('W&B is not enabled!')

# overwrite the default config with previous run config
config.mode = 'test'
run_config = OmegaConf.load(f'{local_path}/usr_config.yaml')
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f'{local_path}/usr_config_test.yaml')
    print(f'Overwrite the previous run config with new run config.')
config = set_config_run(config, "test")

device = torch.device("cuda")


# my_model = seg_model.ShapeNetModel(config)
# my_model = seg_model.ShapeNetModel_fps(config)
my_model = seg_model.ShapeNetModel_fpsknn(config)
my_model = my_model.to(device)
my_model.eval()

# x = torch.randn((8, 3, 2048)).to(device)
# category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)
# flops = FlopCountAnalysis(my_model, (x, category_id))
# print(f"FLOPs: {flops.total()}")

summary(
    my_model,
    input_size=((8, 3, 2048), (8, 16, 1)),
    col_names=["input_size", "output_size", "num_params", "mult_adds"],
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fuhaodsst (apes3). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hfu\.netrc


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Mult-Adds
ShapeNetModel_fpsknn                          [8, 3, 2048]              [8, 50, 2048]             --                        --
├─STN: 1-1                                    [8, 6, 2048, 32]          [8, 3, 3]                 --                        --
│    └─Sequential: 2-1                        [8, 6, 2048, 32]          [8, 64, 2048, 32]         --                        --
│    │    └─Conv2d: 3-1                       [8, 6, 2048, 32]          [8, 64, 2048, 32]         384                       201,326,592
│    │    └─BatchNorm2d: 3-2                  [8, 64, 2048, 32]         [8, 64, 2048, 32]         128                       1,024
│    │    └─LeakyReLU: 3-3                    [8, 64, 2048, 32]         [8, 64, 2048, 32]         --                        --
│    └─Sequential: 2-2                        [8, 64, 2048, 32]         [8, 128, 2048, 32]  

In [ ]:
from models import seg_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary


subprocess.run('nvidia-smi', shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load('configs/default.yaml')
cmd_config = {
    'usr_config': 'configs/seg_boltzmannT01_bin6.yaml',
    'datasets': 'shapenet_AnTao350M',
    'wandb': {'name': '2024_04_16_01_26_Shapenet_Token_Std_logmean_1'},
    'test': {'ddp': {'which_gpu': [3]}}
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f'configs/datasets/{config.datasets}.yaml')
dataset_config = OmegaConf.create({'datasets': dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(f'{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest')
    if config.test.suffix.enable:
        local_path = f'./artifacts/{config.wandb.name}_{config.test.suffix.remark}'
    else:
        local_path = f'./artifacts/{config.wandb.name}'
    # artifact.download(root=local_path)
else:
    raise ValueError('W&B is not enabled!')

# overwrite the default config with previous run config
config.mode = 'test'
run_config = OmegaConf.load(f'{local_path}/usr_config.yaml')
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f'{local_path}/usr_config_test.yaml')
    print(f'Overwrite the previous run config with new run config.')
config = set_config_run(config, "test")

device = torch.device("cuda")


my_model = seg_model.ShapeNetModel(config)
my_model = my_model.to(device)
my_model.eval()
total_time=0

for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)

        torch.cuda.synchronize()
        start_time = time.time()       

        my_model(x, category_id)        
        torch.cuda.synchronize()
        total_time += time.time() - start_time

my_model = seg_model.ShapeNetModel_inference_time(config)
my_model = my_model.to(device)
my_model.eval()
inference_time_ds_1 = 0
inference_time_ds_2 = 0

for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)
        preds, time_list = my_model(x, category_id)
        inference_time_ds_1 += time_list[0]
        inference_time_ds_2 += time_list[1]

print(total_time )
print(inference_time_ds_1 )
print(inference_time_ds_2 )

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hfu\.netrc
  0%|          | 0/359 [00:01<?, ?it/s]


NotImplementedError: 

In [ ]:
from models import seg_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary


subprocess.run("nvidia-smi", shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load("configs/default.yaml")
cmd_config = {
    "usr_config": "configs/seg_boltzmannT01_bin6.yaml",
    "datasets": "shapenet_AnTao350M",
    "wandb": {"name": "2024_04_16_01_26_Shapenet_Token_Std_logmean_1"},
    "test": {"ddp": {"which_gpu": [3]}},
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f"configs/datasets/{config.datasets}.yaml")
dataset_config = OmegaConf.create({"datasets": dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(
        f"{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest"
    )
    if config.test.suffix.enable:
        local_path = f"./artifacts/{config.wandb.name}_{config.test.suffix.remark}"
    else:
        local_path = f"./artifacts/{config.wandb.name}"
    artifact.download(root=local_path)
else:
    raise ValueError("W&B is not enabled!")

# overwrite the default config with previous run config
config.mode = "test"
run_config = OmegaConf.load(f"{local_path}/usr_config.yaml")
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f"{local_path}/usr_config_test.yaml")
    print(f"Overwrite the previous run config with new run config.")
config = set_config_run(config, "test")

device = torch.device("cuda")

my_model = seg_model.ShapeNetModel_fps(config)
my_model = my_model.to(device)
my_model.eval()
total_time = 0
for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)

        torch.cuda.synchronize()
        start_time = time.time()
        my_model(x, category_id)
        torch.cuda.synchronize()
        total_time += time.time() - start_time

my_model = seg_model.ShapeNetModel_inference_time_fps(config)
my_model = my_model.to(device)
my_model.eval()
inference_time_ds_1 = 0
inference_time_ds_2 = 0
for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)
        preds, time_list = my_model(x, category_id)
        inference_time_ds_1 += time_list[0]
        inference_time_ds_2 += time_list[1]

print(total_time)
print(inference_time_ds_1)
print(inference_time_ds_2)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fuhaodsst (apes3). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hfu\.netrc
wandb:   19 of 19 files downloaded.  


In [ ]:
from models import cls_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary


subprocess.run('nvidia-smi', shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load('configs/default.yaml')
cmd_config = {
    "usr_config": "configs/boltzmannT0102.yaml",
    "datasets": "modelnet_AnTao420M",
    "wandb": {"name": "2024_04_28_07_27_Modelnet_Token_Std_boltzmann_T01_bin6_4"},
    "test": {"ddp": {"which_gpu": [0]}},
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f'configs/datasets/{config.datasets}.yaml')
dataset_config = OmegaConf.create({'datasets': dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(
        f"{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest"
    )
    if config.test.suffix.enable:
        local_path = f"./artifacts/{config.wandb.name}_{config.test.suffix.remark}"
    else:
        local_path = f"./artifacts/{config.wandb.name}"
    artifact.download(root=local_path)
else:
    raise ValueError("W&B is not enabled!")

# overwrite the default config with previous run config
config.mode = "test"
run_config = OmegaConf.load(f"{local_path}/usr_config.yaml")
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f"{local_path}/usr_config_test.yaml")
    print(f"Overwrite the previous run config with new run config.")
config = set_config_run(config, "test")

device = torch.device("cuda")

my_model = cls_model.ModelNetModel(config)
my_model.eval()
my_model = my_model.to(device)

total_time = 0
for i in tqdm(range(359)):
    with torch.no_grad():  # x.shape == (B, 3, N)
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        samples = torch.randn((8, 3, 2048)).to(device)

        torch.cuda.synchronize()
        start_time = time.time()
        my_model(samples)
        torch.cuda.synchronize()
        total_time += time.time() - start_time
print(total_time)

my_model = cls_model.ModelNetModel_inference_time(config)
my_model = my_model.to(device)
my_model.eval()
inference_time_ds_1 = 0
inference_time_ds_2 = 0

for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        preds, time_list = my_model(samples)
        inference_time_ds_1 += time_list[0]
        inference_time_ds_2 += time_list[1]


print(inference_time_ds_1)
print(inference_time_ds_2)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fuhaodsst (apes3). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hfu\_netrc
wandb:   19 of 19 files downloaded.  
100%|██████████| 359/359 [00:20<00:00, 17.90it/s]


19.82380700111389
4.395751714706421
2.3908379077911377


In [ ]:
from models import cls_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary


subprocess.run("nvidia-smi", shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load("configs/default.yaml")
cmd_config = {
    "usr_config": "configs/boltzmannT0102.yaml",
    "datasets": "modelnet_AnTao420M",
    "wandb": {"name": "2024_04_28_07_27_Modelnet_Token_Std_boltzmann_T01_bin6_4"},
    "test": {"ddp": {"which_gpu": [0]}},
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f"configs/datasets/{config.datasets}.yaml")
dataset_config = OmegaConf.create({"datasets": dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(
        f"{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest"
    )
    if config.test.suffix.enable:
        local_path = f"./artifacts/{config.wandb.name}_{config.test.suffix.remark}"
    else:
        local_path = f"./artifacts/{config.wandb.name}"
    artifact.download(root=local_path)
else:
    raise ValueError("W&B is not enabled!")

# overwrite the default config with previous run config
config.mode = "test"
run_config = OmegaConf.load(f"{local_path}/usr_config.yaml")
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f"{local_path}/usr_config_test.yaml")
    print(f"Overwrite the previous run config with new run config.")
config = set_config_run(config, "test")

device = torch.device("cuda")

my_model = cls_model.ModelNetModel_fps(config)
my_model.eval()
my_model = my_model.to(device)

total_time = 0
for i in tqdm(range(359)):
    with torch.no_grad():  # x.shape == (B, 3, N)
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        samples = torch.randn((8, 3, 2048)).to(device)

        torch.cuda.synchronize()
        start_time = time.time()
        my_model(samples)
        torch.cuda.synchronize()
        total_time += time.time() - start_time
print(total_time)

my_model = cls_model.ModelNetModel_fps_inference_time(config)
my_model = my_model.to(device)
my_model.eval()
inference_time_ds_1 = 0
inference_time_ds_2 = 0

for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        preds, time_list = my_model(samples)
        inference_time_ds_1 += time_list[0]
        inference_time_ds_2 += time_list[1]


print(inference_time_ds_1)
print(inference_time_ds_2)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hfu\_netrc
wandb:   19 of 19 files downloaded.  
100%|██████████| 359/359 [03:49<00:00,  1.56it/s]


229.27262926101685
143.62903594970703
68.53185868263245


In [ ]:
from models import cls_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary


subprocess.run("nvidia-smi", shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load("configs/default.yaml")
cmd_config = {
    "usr_config": "configs/boltzmannT0102.yaml",
    "datasets": "modelnet_AnTao420M",
    "wandb": {"name": "2024_04_28_07_27_Modelnet_Token_Std_boltzmann_T01_bin6_4"},
    "test": {"ddp": {"which_gpu": [0]}},
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f"configs/datasets/{config.datasets}.yaml")
dataset_config = OmegaConf.create({"datasets": dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(
        f"{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest"
    )
    if config.test.suffix.enable:
        local_path = f"./artifacts/{config.wandb.name}_{config.test.suffix.remark}"
    else:
        local_path = f"./artifacts/{config.wandb.name}"
    # artifact.download(root=local_path)
else:
    raise ValueError("W&B is not enabled!")

# overwrite the default config with previous run config
config.mode = "test"
run_config = OmegaConf.load(f"{local_path}/usr_config.yaml")
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f"{local_path}/usr_config_test.yaml")
    print(f"Overwrite the previous run config with new run config.")
config = set_config_run(config, "test")

device = torch.device("cuda")

my_model = cls_model.ModelNetModel_fpsknn(config)
my_model.eval()
my_model = my_model.to(device)

total_time = 0
for i in tqdm(range(359)):
    with torch.no_grad():  # x.shape == (B, 3, N)
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        samples = torch.randn((8, 3, 2048)).to(device)

        torch.cuda.synchronize()
        start_time = time.time()
        my_model(samples)
        torch.cuda.synchronize()
        total_time += time.time() - start_time
print(total_time)

my_model = cls_model.ModelNetModel_fpsknn_inference_time(config)
my_model = my_model.to(device)
my_model.eval()
inference_time_ds_1 = 0
inference_time_ds_2 = 0

for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        preds, time_list = my_model(samples)
        inference_time_ds_1 += time_list[0]
        inference_time_ds_2 += time_list[1]


print(inference_time_ds_1)
print(inference_time_ds_2)

KeyboardInterrupt: 

In [ ]:
from models import seg_model
import subprocess
from omegaconf import OmegaConf
import wandb
from utils.check_config import set_config_run
import torch
import time
from tqdm import tqdm
from torchinfo import summary


subprocess.run("nvidia-smi", shell=True, text=True, stdout=None, stderr=subprocess.PIPE)
config = OmegaConf.load("configs/default.yaml")
cmd_config = {
    "usr_config": "configs/seg_boltzmannT01_bin6.yaml",
    "datasets": "shapenet_AnTao350M",
    "wandb": {"name": "2024_04_16_01_26_Shapenet_Token_Std_logmean_1"},
    "test": {"ddp": {"which_gpu": [3]}},
}
config = OmegaConf.merge(config, OmegaConf.create(cmd_config))

dataset_config = OmegaConf.load(f"configs/datasets/{config.datasets}.yaml")
dataset_config = OmegaConf.create({"datasets": dataset_config})
config = OmegaConf.merge(config, dataset_config)

# get test configurations
if config.usr_config:
    test_config = OmegaConf.load(config.usr_config)
    config = OmegaConf.merge(config, test_config)

# download artifacts
if config.wandb.enable:
    wandb.login(key=config.wandb.api_key)
    api = wandb.Api()
    artifact = api.artifact(
        f"{config.wandb.entity}/{config.wandb.project}/{config.wandb.name}:latest"
    )
    if config.test.suffix.enable:
        local_path = f"./artifacts/{config.wandb.name}_{config.test.suffix.remark}"
    else:
        local_path = f"./artifacts/{config.wandb.name}"
    # artifact.download(root=local_path)
else:
    raise ValueError("W&B is not enabled!")

# overwrite the default config with previous run config
config.mode = "test"
run_config = OmegaConf.load(f"{local_path}/usr_config.yaml")
if not config.test.suffix.enable:
    config = OmegaConf.merge(config, run_config)
else:
    OmegaConf.save(config, f"{local_path}/usr_config_test.yaml")
    print(f"Overwrite the previous run config with new run config.")
config = set_config_run(config, "test")

device = torch.device("cuda")

my_model = seg_model.ShapeNetModel_fpsknn(config)
my_model = my_model.to(device)
my_model.eval()
total_time = 0
for i in tqdm(range(359)):  # 359
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)

        torch.cuda.synchronize()
        start_time = time.time()
        my_model(x, category_id)
        torch.cuda.synchronize()
        total_time += time.time() - start_time

my_model = seg_model.ShapeNetModel_inference_time_fpsknn(config)
my_model = my_model.to(device)
my_model.eval()
inference_time_ds_1 = 0
inference_time_ds_2 = 0
for i in tqdm(range(359)):
    with torch.no_grad():
        # x.shape == (B, 3, N)  category_id.shape == (B, 16, 1)
        x = torch.randn((8, 3, 2048)).to(device)
        category_id = torch.randint(0, 1, (8, 16, 1)).float().to(device)
        preds, time_list = my_model(x, category_id)
        inference_time_ds_1 += time_list[0]
        inference_time_ds_2 += time_list[1]

print(total_time)
print(inference_time_ds_1)
print(inference_time_ds_2)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fuhaodsst (apes3). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hfu\.netrc
